### Import functions

In [1]:
# Import packages
import argparse
import os
import warnings
import rasterio as rio
import pandas as pd
import fiona
import matplotlib.pyplot as plt
import numpy.ma as ma
import numpy as np
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping, box
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import rasterio
from rasterio.transform import from_origin
import sys
sys.path.append(r'C:\Users\SkosanaT\OneDrive - Stellenbosch University\MAPWAPS\DataChapter1\Code\umzimAOI\modules')
from emit_tools import emit_xarray

warnings.simplefilter('ignore')

### Scene 1

In [2]:
# Specify the file path
emit_fp = r'F:\emit\EMIT\FEB_S1\EMIT_L2A_RFL_001_20240229T101944_2406007_016.nc'

# open emit scene and orthoretify it.
emit_ds = emit_xarray(emit_fp, ortho=True)
emit_ds


<xarray.Dataset> Size: 5GB
Dimensions:           (latitude: 2036, longitude: 2116, wavelengths: 285)
Coordinates:
  * wavelengths       (wavelengths) float32 1kB 381.0 388.4 ... 2.493e+03
    fwhm              (wavelengths) float32 1kB ...
    good_wavelengths  (wavelengths) float32 1kB ...
  * latitude          (latitude) float64 16kB -30.26 -30.27 ... -31.37 -31.37
  * longitude         (longitude) float64 17kB 28.15 28.15 28.15 ... 29.3 29.3
    elev              (latitude, longitude) float32 17MB nan nan nan ... nan nan
    spatial_ref       int32 4B 0
Data variables:
    reflectance       (latitude, longitude, wavelengths) float32 5GB nan ... nan
Attributes: (12/40)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [ 2.81535207e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20240229T101944_24060...
    Orthorectified:                    True

In [3]:
#Define output path
output_path = r'F:\emit\EMIT\FEB_S1\EMIT_L2A_TIF_001_20240229T101944_2406007_016.tif'

# Get the array data representing reflectance
reflectance_array = emit_ds['reflectance'].values

# Define metadata for the raster data
metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': None,
    'width': emit_ds.sizes['longitude'],
    'height': emit_ds.sizes['latitude'],
    'count': emit_ds.sizes['wavelengths'],
    'compress': 'lzw',  # Compression method ((Lempel-Ziv-Welch),
    'crs': 'EPSG:4326',
    'transform': emit_ds['reflectance'].rio.transform()  # Use existing transformation
}
# Reshape the array to match the expected dimensions for writing
reflectance_array = np.transpose(reflectance_array, axes=(2, 0, 1))  # Move the band dimension to the first axis

# Write the array to a raster file
with rasterio.open(output_path, 'w', **metadata) as dst:
    dst.write(reflectance_array)

# Write the array to a raster file

with rasterio.open(output_path, 'w', **metadata) as dst:
    dst.write(reflectance_array)